In [164]:
# Import libraries and dependencies
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import hvplot.pandas


In [165]:
current_portfolio = input("what stocks are in your portfolio? enter tickers with all cap and no spaces seperated with commas")
#taking in users current assets

what stocks are in your portfolio? enter tickers with all cap and no spaces seperated with commas AGG,SPY,FB


In [166]:
asset_list = current_portfolio.split(",")
print(asset_list)
#confirming previous cell

['AGG', 'SPY', 'FB']


In [167]:
# Load .env environment variables
load_dotenv()

True

In [168]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [169]:
# Set the tickers as object of input function, could just pass asset list into get barset function
#but i thought this was clearer

# Set timeframe to 1D 
timeframe = "1D"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2017-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

In [170]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
portfolio_df = alpaca.get_barset(tickers, timeframe, start=start_date, end=end_date, limit=1000).df

# Review the first 5 rows of the Alpaca DataFrame
portfolio_df.tail()

AGG                                       \
                             open    high       low    close    volume   
time                                                                     
2020-08-03 00:00:00-04:00  119.37  119.40  119.1903  119.400  24615464   
2020-08-04 00:00:00-04:00  119.42  119.63  119.4200  119.630  21512268   
2020-08-05 00:00:00-04:00  119.39  119.49  119.3100  119.400  34239733   
2020-08-06 00:00:00-04:00  119.62  119.73  119.5300  119.580   9021388   
2020-08-07 00:00:00-04:00  119.66  119.73  119.3950  119.445   8830420   

                               FB                                       \
                             open      high      low   close    volume   
time                                                                     
2020-08-03 00:00:00-04:00  252.65  255.4000  250.330  251.84  21498775   
2020-08-04 00:00:00-04:00  251.56  252.8000  247.430  249.83  15733319   
2020-08-05 00:00:00-04:00  250.19  252.2698  247.770  249.13  11988089   
2020-08-06 00:00:00-04:00  249.04  266.6000  248.665  265.28  43995059   
2020-08-07 00:00:00-04:00  264.08  278.8900  263.430  268.50  71746335   

                                SPY                                    
                               open    high     low   close    volume  
time                                                                   
2020-08-03 00:00:00-04:00  328.3200  329.62  327.73  328.76  89861354  
2020-08-04 00:00:00-04:00  327.8600  330.06  327.86  330.03  73684427  
2020-08-05 00:00:00-04:00  331.4700  332.39  331.18  332.06  72941562  
2020-08-06 00:00:00-04:00  331.4799  334.46  331.13  334.31  76988181  
2020-08-07 00:00:00-04:00  333.2800  334.88  332.30  334.55  98710236

In [188]:
#taking the std and yearly returns as is own dataframe
#the data here had to be cleaned as is seen in the .value sfunction and .str[] function. This was necessary due to 
#the data having to be converting back and forth from a series to a dataframe 

std_list = []
yearly_return = []

for asset in asset_list:
    
    
    daily_return = portfolio_df[asset][["close"]].pct_change() *100
    yearly_var = (daily_return.mean() * 252).values
    
    
    yearly_return.append(yearly_var)
    
    std_var = daily_return.std().values
    
    std_list.append(std_var)
    

print(yearly_var)
print(std_var)



ef_df = pd.DataFrame({'Ticker': asset_list, 'STD' : std_list, 'Yearly Return' : yearly_return})
ef_df['STD'] = ef_df['STD'].str[0]
ef_df['Yearly Return'] = ef_df['Yearly Return'].str[0]
ef_df.head()

[21.52654879]
[2.28432513]


,Ticker,STD,Yearly Return
0,AGG,0.350569,2.997449
1,SPY,1.415552,12.541743
2,FB,2.284325,21.526549


In [189]:
ef_df.hvplot.scatter(x='STD', y='Yearly Return', hover_cols = ['Ticker'])
#plotting the current simplified efficient frontier, not seen for the user just for the programmer

:Scatter   [STD]   (Yearly Return,Ticker)

In [205]:
#second prompt for user to put on the stocks they are looking to buy 
new_portfolio = input("what stocks are looking to buy? enter tickers with all cap and no spaces seperated with commas")


what stocks are looking to buy? enter tickers with all cap and no spaces seperated with commas MSFT,AAPL,GOOG


In [207]:
new_asset_list = new_portfolio.split(",")
print(new_asset_list)
#confirming previous cell

['MSFT', 'AAPL', 'GOOG']


In [208]:
#here i just pass the the prompt object in directly, new_asset_list

new_asset_df = alpaca.get_barset(new_asset_list, timeframe, start=start_date, end=end_date, limit=1000).df
new_asset_df.head()

AAPL                                        \
                             open      high       low   close    volume   
time                                                                      
2017-08-07 00:00:00-04:00  157.06  158.9200  156.6701  158.79  17348586   
2017-08-08 00:00:00-04:00  158.61  161.8300  158.2700  160.10  31403180   
2017-08-09 00:00:00-04:00  159.26  161.2700  159.1100  161.10  22294726   
2017-08-10 00:00:00-04:00  159.90  160.0000  154.6300  155.32  32603524   
2017-08-11 00:00:00-04:00  156.60  158.5728  156.0700  157.49  22197629   

                             GOOG                                       MSFT  \
                             open     high       low   close   volume   open   
time                                                                           
2017-08-07 00:00:00-04:00  929.06  931.700  926.5000  928.85   637172  72.80   
2017-08-08 00:00:00-04:00  927.09  935.814  925.6095  926.76   611275  72.09   
2017-08-09 00:00:00-04:00  920.61  925.980  917.2501  922.90   753312  72.25   
2017-08-10 00:00:00-04:00  917.55  919.260  906.1300  907.23  1012901  71.95   
2017-08-11 00:00:00-04:00  907.97  917.780  905.5800  914.45   710452  71.57   

                                                          
                            high    low  close    volume  
time                                                      
2017-08-07 00:00:00-04:00  72.90  72.26  72.40  14859602  
2017-08-08 00:00:00-04:00  73.13  71.75  72.78  16958136  
2017-08-09 00:00:00-04:00  72.51  72.05  72.45  16809135  
2017-08-10 00:00:00-04:00  72.19  71.35  71.42  18839204  
2017-08-11 00:00:00-04:00  72.70  71.28  72.51  17762519

In [216]:
new_std_list = []
new_yearly_return = []
#repeating first steps for second input 



for asset in new_asset_list:
    
    
    newdaily_return = new_asset_df[asset][["close"]].pct_change() *100
    newyearly_var = (newdaily_return.mean() * 252).values
    
    
    new_yearly_return.append(newyearly_var)
    
    new_std_var = newdaily_return.std().values
    
    new_std_list.append(new_std_var)

    
    
    
newef_df = pd.DataFrame({'Ticker': new_asset_list, 'STD' : new_std_list, 'Yearly Return' : new_yearly_return})
newef_df['STD'] = newef_df['STD'].str[0]
newef_df['Yearly Return'] = newef_df['Yearly Return'].str[0]
newef_df.head()

,Ticker,STD,Yearly Return
0,MSFT,1.955767,40.719794
1,AAPL,2.075049,39.754989
2,GOOG,1.854781,20.185514


In [219]:
avg_std1 = (ef_df['STD'].mean()).values
avg_yr1 = (ef_df['Yearly Return'].mean()).values
sr_current = avg_yr1 / avg_std1
#creating threshold for ew asset to surpass in order to be accpeted in the portfolio, lableed as sr_curretn for current 
#current shapre ratio 

new_sr_list = []
for i in newef_df:
    a = newef_df['STD'] / newef_df['Yearly Return']
    new_sr_list.append(a)
#attempted conditional statement. I think I will attepmt this action in the for loop on the previous cell so 
#i can keep the asset names instead of seeing a list of integers
#print(threshold)
print(new_sr_list)

[0    0.048030
1    0.052196
2    0.091887
dtype: float64, 0    0.048030
1    0.052196
2    0.091887
dtype: float64, 0    0.048030
1    0.052196
2    0.091887
dtype: float64]


In [221]:
for sr in new_sr_list:
    if sr > sr_current:
        print(f'buy {sr}')
    elif sr == sr_current:
            print(f'you get the same bang for your buck with asset {sr}')
    elif sr < sr_current:
        print(f'dont buy {sr}')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:


new_asset_std_over_yr = newef_df['STD'][asset] / newef_df['Yearly Return'][asset]
std_over_yr_threshold = ef_df['STD'] / ef_df['Yearly Return'][asset]

for asset in new_asset_list
    if newef_df['STD'][asset] / newef_df['Yearly Return'][asset]

In [163]:
 daily_returns = portfolio_df[asset][["close"]].pct_change() *100
yearly_returns = daily_returns.mean() * 252
print( yearly_returns)

close    21.520229
dtype: float64


In [90]:
daily_returns = portfolio_df['SPY'][["close"]].pct_change() *100
print(daily_returns)

                              close
time                               
2017-08-07 00:00:00-04:00       NaN
2017-08-08 00:00:00-04:00 -0.225540
2017-08-09 00:00:00-04:00 -0.024667
2017-08-10 00:00:00-04:00 -1.391417
2017-08-11 00:00:00-04:00  0.131261
...                             ...
2020-08-03 00:00:00-04:00  0.676772
2020-08-04 00:00:00-04:00  0.386300
2020-08-05 00:00:00-04:00  0.615096
2020-08-06 00:00:00-04:00  0.677588
2020-08-07 00:00:00-04:00  0.071790

[757 rows x 1 columns]


In [67]:
result = pd.DataFrame()
for asset in asset_list:
    daily_returns = portfolio_df[asset]["close"].pct_change() *100
    yearly_rerutn = daily_returns.mean() * 252
    asset_std = daily_returns.std
    result.append(daily_returns)
    

result.head()   
    #print(f"The {asset} closing price is ${close_price}")

""


In [79]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
agg_close_price = portfolio_df["AGG"]["close"]

# Print the AGG closing price
agg_close_price_float = (agg_close_price)
print(f"The AGG closing price is ${agg_close_price_float}")

The AGG closing price is $time
2017-08-07 00:00:00-04:00    109.6850
2017-08-08 00:00:00-04:00    109.5810
2017-08-09 00:00:00-04:00    109.6700
2017-08-10 00:00:00-04:00    109.8053
2017-08-11 00:00:00-04:00    109.8850
                               ...   
2020-08-03 00:00:00-04:00    119.4000
2020-08-04 00:00:00-04:00    119.6300
2020-08-05 00:00:00-04:00    119.4000
2020-08-06 00:00:00-04:00    119.5800
2020-08-07 00:00:00-04:00    119.4450
Name: close, Length: 757, dtype: float64


In [80]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
spy_close_price = portfolio_df["SPY"]["close"]

# Print the SPY closing price
spy_close_price_float = (spy_close_price)
print(f"The SPY closing price is ${spy_close_price_float}")

The SPY closing price is $time
2017-08-07 00:00:00-04:00    247.850
2017-08-08 00:00:00-04:00    247.291
2017-08-09 00:00:00-04:00    247.230
2017-08-10 00:00:00-04:00    243.790
2017-08-11 00:00:00-04:00    244.110
                              ...   
2020-08-03 00:00:00-04:00    328.760
2020-08-04 00:00:00-04:00    330.030
2020-08-05 00:00:00-04:00    332.060
2020-08-06 00:00:00-04:00    334.310
2020-08-07 00:00:00-04:00    334.550
Name: close, Length: 757, dtype: float64


In [81]:
# Calculate average daily returns for assets
agg_daily_return = agg_close_price.pct_change().dropna() * 100
print(f"The daily return for AGG is ${agg_daily_return}")

spy_daily_return = spy_close_price.pct_change().dropna() * 100
print(f"The daily return for SPY is ${spy_daily_return}")

The daily return for AGG is $time
2017-08-08 00:00:00-04:00   -0.094817
2017-08-09 00:00:00-04:00    0.081218
2017-08-10 00:00:00-04:00    0.123370
2017-08-11 00:00:00-04:00    0.072583
2017-08-14 00:00:00-04:00   -0.068253
                               ...   
2020-08-03 00:00:00-04:00   -0.133824
2020-08-04 00:00:00-04:00    0.192630
2020-08-05 00:00:00-04:00   -0.192259
2020-08-06 00:00:00-04:00    0.150754
2020-08-07 00:00:00-04:00   -0.112895
Name: close, Length: 756, dtype: float64
The daily return for SPY is $time
2017-08-08 00:00:00-04:00   -0.225540
2017-08-09 00:00:00-04:00   -0.024667
2017-08-10 00:00:00-04:00   -1.391417
2017-08-11 00:00:00-04:00    0.131261
2017-08-14 00:00:00-04:00    0.987260
                               ...   
2020-08-03 00:00:00-04:00    0.676772
2020-08-04 00:00:00-04:00    0.386300
2020-08-05 00:00:00-04:00    0.615096
2020-08-06 00:00:00-04:00    0.677588
2020-08-07 00:00:00-04:00    0.071790
Name: close, Length: 756, dtype: float64


In [82]:
agg_yearly =  agg_daily_return.mean() * 252
agg_yearly

2.9974485108771134

In [83]:
spy_yearly =  spy_daily_return.mean() * 252
spy_yearly

12.541742889850356

In [84]:
agg_std =  agg_daily_return.std() 
agg_std

0.350568819956525

In [85]:
spy_std =  spy_daily_return.std()
spy_std

1.4155516047457433

In [86]:
ticker = ['SPY', 'AGG'] 
std = [agg_std, spy_std]
yearly_return = [spy_yearly, agg_yearly]

In [87]:
ef_df = pd.DataFrame({'Ticker': ticker, 'STD' : std, 'Yearly Return' : yearly_return})
ef_df

,Ticker,STD,Yearly Return
0,SPY,0.350569,12.541743
1,AGG,1.415552,2.997449


In [88]:
ef_df.hvplot.scatter(x='STD', y='Yearly Return')

:Scatter   [STD]   (Yearly Return)